# Reads stats file into a dataframe variable

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BattingStats").getOrCreate()

batting_df = spark.read.csv("2023Stats.csv", header=True, inferSchema=True)

In [2]:
batting_df.show(50)

+---+-----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+-----+----+---+---+---+---+---+---+-----------+---------------+
| Rk|             Name|Age| Tm| Lg|  G| PA| AB|  R|  H| 2B| 3B| HR|RBI| SB| CS| BB| SO|   BA|  OBP|  SLG|  OPS|OPS+| TB|GDP|HBP| SH| SF|IBB|Pos Summary|Name-additional|
+---+-----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+-----+----+---+---+---+---+---+---+-----------+---------------+
|  1|       CJ Abrams*| 22|WSN| NL|151|614|563| 83|138| 28|  6| 18| 64| 47|  4| 32|118|0.245|  0.3|0.412|0.712|  95|232|  7| 13|  3|  3|  2|       *6/H|      abramcj01|
|  2|       José Abreu| 36|HOU| AL|141|594|540| 62|128| 23|  1| 18| 90|  0|  1| 42|130|0.237|0.296|0.383| 0.68|  87|207| 16|  6|  0|  6|  1|       *3/D|      abreujo02|
|  3|    Wilyer Abreu*| 24|BOS| AL| 28| 85| 76| 10| 24|  6|  0|  2| 14|  3|  1|  9| 23|0.316|0.388|0.474|0.862| 132| 36|  0|  0|  0|  0|  0|     87/H9D|   

In [3]:
cols_to_del = ["Rk", "SB", "CS", "TB", "GDP", "HBP", "SH", "SF", "IBB", "Name-additional", "Pos Summary"]
batting_df = batting_df.drop(*cols_to_del)

In [4]:
nl_batting = batting_df.filter(batting_df["Lg"] == "NL")

In [5]:
nl_batting = nl_batting.dropna(how='any')

In [6]:
nl_batting.show()

+-----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+-----+----+
|             Name|Age| Tm| Lg|  G| PA| AB|  R|  H| 2B| 3B| HR|RBI| BB| SO|   BA|  OBP|  SLG|  OPS|OPS+|
+-----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+-----+----+
|       CJ Abrams*| 22|WSN| NL|151|614|563| 83|138| 28|  6| 18| 64| 32|118|0.245|  0.3|0.412|0.712|  95|
| Ronald Acuna Jr.| 25|ATL| NL|159|735|643|149|217| 35|  4| 41|106| 80| 84|0.337|0.416|0.596|1.012| 168|
|     Willy Adames| 27|MIL| NL|149|638|553| 73|120| 29|  2| 24| 80| 71|165|0.217| 0.31|0.407|0.717|  95|
|      Riley Adams| 27|WSN| NL| 44|158|143|  8| 39| 13|  2|  4| 21| 11| 45|0.273|0.331|0.476|0.807| 120|
|  Ehire Adrianza#| 33|ATL| NL|  5| 11| 10|  0|  0|  0|  0|  0|  0|  1|  4|  0.0|0.091|  0.0|0.091| -72|
|       Nick Ahmed| 33|ARI| NL| 72|210|198| 14| 42| 10|  1|  2| 17| 12| 52|0.212|0.257|0.303| 0.56|  53|
|    Ozzie Albies#| 26|ATL| NL|148|660|596| 96|167| 30|

In [7]:
# Handling Categorical Features
from pyspark.ml.feature import StringIndexer

In [8]:
indexer = StringIndexer(inputCol="Tm", outputCol="Tm_indexed")
nl_batting_indexed = indexer.fit(nl_batting).transform(nl_batting)
nl_batting_indexed.show(truncate=False)

+-----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+-----+----+----------+
|Name             |Age|Tm |Lg |G  |PA |AB |R  |H  |2B |3B |HR |RBI|BB |SO |BA   |OBP  |SLG  |OPS  |OPS+|Tm_indexed|
+-----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+-----+----+----------+
|CJ Abrams*       |22 |WSN|NL |151|614|563|83 |138|28 |6  |18 |64 |32 |118|0.245|0.3  |0.412|0.712|95  |11.0      |
|Ronald Acuna Jr. |25 |ATL|NL |159|735|643|149|217|35 |4  |41 |106|80 |84 |0.337|0.416|0.596|1.012|168 |12.0      |
|Willy Adames     |27 |MIL|NL |149|638|553|73 |120|29 |2  |24 |80 |71 |165|0.217|0.31 |0.407|0.717|95  |4.0       |
|Riley Adams      |27 |WSN|NL |44 |158|143|8  |39 |13 |2  |4  |21 |11 |45 |0.273|0.331|0.476|0.807|120 |11.0      |
|Ehire Adrianza#  |33 |ATL|NL |5  |11 |10 |0  |0  |0  |0  |0  |0  |1  |4  |0.0  |0.091|0.0  |0.091|-72 |12.0      |
|Nick Ahmed       |33 |ARI|NL |72 |210|198|14 |42 |10 |1  |2  |17 |12 |5

In [52]:
from pyspark.ml.feature import VectorAssembler
featAssemb = VectorAssembler(inputCols=['AB', 'H', 'SLG'], outputCol='Independent Features')
output = featAssemb.transform(nl_batting_indexed)

In [53]:
finalized_data = output.select('Independent Features', 'HR')

In [54]:
finalized_data.show()

+--------------------+---+
|Independent Features| HR|
+--------------------+---+
| [563.0,138.0,0.412]| 18|
| [643.0,217.0,0.596]| 41|
| [553.0,120.0,0.407]| 24|
|  [143.0,39.0,0.476]|  4|
|      [10.0,0.0,0.0]|  0|
|  [198.0,42.0,0.303]|  2|
| [596.0,167.0,0.513]| 33|
|    [31.0,5.0,0.387]|  1|
|    [15.0,1.0,0.133]|  0|
| [568.0,123.0,0.504]| 46|
|  [159.0,36.0,0.289]|  2|
|  [382.0,80.0,0.437]| 25|
|     [9.0,2.0,0.222]|  0|
|  [131.0,28.0,0.359]|  5|
|  [318.0,72.0,0.368]|  9|
|   [84.0,21.0,0.476]|  4|
|    [59.0,8.0,0.288]|  3|
|  [488.0,129.0,0.42]| 17|
| [560.0,149.0,0.459]| 26|
| [574.0,203.0,0.469]| 10|
+--------------------+---+
only showing top 20 rows



In [55]:
from pyspark.ml.regression import LinearRegression
#train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='HR')
regressor = regressor.fit(train_data)

In [56]:
coefficients = regressor.coefficients

In [57]:
intercept = regressor.intercept

In [58]:
results = regressor.evaluate(test_data)

In [34]:
results.predictions.columns

['Independent Features', 'HR', 'prediction']

In [59]:
results.predictions.show(100)

+--------------------+---+--------------------+
|Independent Features| HR|          prediction|
+--------------------+---+--------------------+
|       [1.0,0.0,0.0]|  0|  -4.579095070628332|
|       [1.0,1.0,1.0]|  0|   6.209868987349413|
|       [5.0,1.0,0.2]|  0| -2.2636104195315876|
|       [6.0,2.0,0.5]|  0|  1.0164827949188187|
|       [8.0,4.0,0.5]|  0|  1.1080680336511266|
|     [8.0,4.0,0.625]|  0|  2.4556932816028985|
|       [9.0,0.0,0.0]|  0|  -4.276450710607652|
|     [9.0,1.0,0.111]|  0|   -3.07179741606291|
|      [10.0,2.0,0.2]|  0|  -2.066495620155094|
|      [14.0,0.0,0.0]|  0|  -4.087297985594728|
|    [14.0,5.0,0.429]|  0|  0.5775622371935984|
|    [18.0,3.0,0.222]|  0| -1.5187071421313338|
|    [18.0,8.0,0.778]|  2|  4.5153403325759935|
|    [19.0,1.0,0.053]|  0|  -3.318790081086683|
|     [20.0,2.0,0.15]|  0|  -2.227240269309954|
|    [23.0,6.0,0.348]|  0|0.052738055907683545|
|    [24.0,4.0,0.208]|  0| -1.4346958255228537|
|    [28.0,8.0,0.464]|  1|  1.5084111597

In [60]:
r2, mae, mse = results.r2,results.meanAbsoluteError,results.meanSquaredError

In [61]:
coefficients, intercept, r2, mae, mse

(DenseVector([0.0378, 0.008, 10.781]),
 -4.616925615630917,
 0.7872760164229433,
 2.674185335705098,
 15.923169692523267)

In [63]:
results.predictions

+--------------------+---+--------------------+
|Independent Features| HR|          prediction|
+--------------------+---+--------------------+
|       [1.0,0.0,0.0]|  0|  -4.579095070628332|
|       [1.0,1.0,1.0]|  0|   6.209868987349413|
|       [5.0,1.0,0.2]|  0| -2.2636104195315876|
|       [6.0,2.0,0.5]|  0|  1.0164827949188187|
|       [8.0,4.0,0.5]|  0|  1.1080680336511266|
|     [8.0,4.0,0.625]|  0|  2.4556932816028985|
|       [9.0,0.0,0.0]|  0|  -4.276450710607652|
|     [9.0,1.0,0.111]|  0|   -3.07179741606291|
|      [10.0,2.0,0.2]|  0|  -2.066495620155094|
|      [14.0,0.0,0.0]|  0|  -4.087297985594728|
|    [14.0,5.0,0.429]|  0|  0.5775622371935984|
|    [18.0,3.0,0.222]|  0| -1.5187071421313338|
|    [18.0,8.0,0.778]|  2|  4.5153403325759935|
|    [19.0,1.0,0.053]|  0|  -3.318790081086683|
|     [20.0,2.0,0.15]|  0|  -2.227240269309954|
|    [23.0,6.0,0.348]|  0|0.052738055907683545|
|    [24.0,4.0,0.208]|  0| -1.4346958255228537|
|    [28.0,8.0,0.464]|  1|  1.5084111597

In [66]:
type(results)

pyspark.ml.regression.LinearRegressionSummary

In [67]:
import pandas as pd

In [70]:
summary_df = pd.DataFrame(list(zip(results.coefficientStandardErrors, results.tValues)),
                           columns=["Coefficient Standard Errors", "T Values"],
                           index=results.features)

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
excel_writer = pd.ExcelWriter("linear_regression_summary.xlsx", engine="xlsxwriter")
summary_df.to_excel(excel_writer, sheet_name="Summary")
excel_writer.save()